### ✅ 학습 목표
- 베스트 모델 저장하기 / 불러오기
- 학습 중단 기능
- 과적합 방지 - Dropout

### ✅ 베스트 모델 저장하기 / 학습 중단

- 📌 손글씨 이미지 데이터셋
    - 훈련 데이터 6만개, 테스트 데이터 1만개
    - 0 ~ 9까지 숫자 이미지에 대한 라벨링

In [ ]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, X_test.shape

- 📌 데이터 확인

In [ ]:
import matplotlib.pyplot as plt

print(y_train[10])

plt.imshow(X_train[10], cmap='gray')
plt.show()

- 📌 이미지(2차원) -> Dense(1차원) 변환

In [ ]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

X_train.shape, X_test.shape

- 📌 스케일링 : 0 ~ 255 -> 0.0 ~ 1.0

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train

- 📌 라벨 데이터는 원핫 인코딩(다진 분류)

In [ ]:
from keras.utils import to_categorical

y_train_en = to_categorical(y_train)
y_test_en = to_categorical(y_test)

y_train_en.shape, y_test_en.shape

- 📌 신경망 설계


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model1 = Sequential()

# input_dim : 입력 데이터의 크기 (X_train의 크기)

model1.add(Dense(512, activation='relu', input_dim=28*28))

# 10 -> 라벨 데이터의 크기 (y_train의 크기)

model1.add(Dense(10, activation='softmax'))

model1.summary()

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy -> 다중분류

- 📌 베스트 모델 저장, 학습 중단 설정
    - 베스트 모델 : 학습 중에서 설정한 기준 값이 가장 좋은 모델을 저장 (ModelCheckPoint())
    - 학습 중단 : 학습 중에 설정한 기준 값이 만족하는 경우에 학습을 중지 시키는 기능 (EaryStopping())

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Deep Learning

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os

# 모델을 저장할 폴더

model_dir = './model/'

# 만약 해당 폴더가 없으면 생성

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# {epoch:02d} : epoch 변수의 값을 2자리 정수로 설정
# {loss:.4f} : loss 변수의 값을 소수점 4째 자리까지로 설정
# {val_loss:.4f} : val_loss 변수의 값을 소수점 4째 자리까지로 설정
# 파일명 예시 : model1_20_0.526_0.406.hdf5

file_name = model_dir + "model1_{epoch:02d}_{loss:.4f}_{val_loss:.4f}.hdf5"

# 베스트 모델 저장 설정
# monitor : 베스트 모델을 결정하는 기준값
# save_best_only=True : monitor에서 설정한 기준값이 더 나아지는 경우에만 저장

mc = ModelCheckpoint(file_name, monitor='val_loss', save_best_only=True) # val_loss -> 과대적합이 안걸리는 가장 바람직한 방법

# 학습 중단 기능 : 얼마만큼 학습 횟수를 설정할지 알 수 없기 때문에 학습이 더 나아지지 않는 경우에 자동으로 중지
# monitor : 학습 중단을 결정하는 기준값 (가능하면 ModelCheckpoint와 동일한 값을 권장)
# patience : 학습이 더 나아지지 않더라도 기다려주는 횟수

es = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# validation_data : 테스트 데이터가 있는 경우
# validation_split : 테스트 데이터가 없는 경우 -> 훈련 데이터에서 비율만큼 가져온다
# callbacks : 반복 시 마다 호출되어 실행

h1 = model1.fit(X_train, y_train_en, epochs=20, batch_size=128, validation_data=(X_test, y_test_en), callbacks=[mc, es])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h1.history['accuracy'], label="train")
plt.plot(h1.history['val_accuracy'], label="test")

plt.legend()
plt.show()

plt.plot(h1.history['loss'], label="train")
plt.plot(h1.history['val_loss'], label="test")

plt.legend()
plt.show()

- 📌 모델 불러오기

In [ ]:
from keras.models import load_model

# model_dir = './model/'

file_name = model_dir + "modelmodel1_01_0.0004_0.0666.hdf5"

model2 = load_model(file_name)

model2.evaluate(X_train, y_train_en)
model2.evaluate(X_test, y_test_en)